In [2]:
from projet import *
from time import *
import numpy as np
import random
from functools import reduce
from copy import deepcopy

# Rapport projet : Jeu séquentiel et stratégie en Python

- NODA Romain n°28720263
- ROIG Thomas n°28708645

## Description

Le projet consiste à tester les performances de différentes stratégies pour un jeu séquentiel, à somme nul et à information complète. 

Dans ce projet on utilise le jeu du Morpion et le jeu des allumettes et 3 stratégies: aléatoire, minimax et optimale pour le jeu des allumettes(Grundy).

## Implémentation

### JeuSequentiel

Les jeux qu'on a implémenté sont hérités à partir de cette classe abstraite

Il contient comme attribut:
- Config : un dictionnaire où on stoque le plateur du jeu, le nombre de coup joué, le joueur courant et l'historique des coups joués
- egalite : une variable booléenne utilisé comme flag pour savoir s'il y a égalité ou pas

et comme fonctions (qui seront redéfini):
- `joueurCourant(C)` : retourne le joueur courant
- `changeJoueur(C)` : change le joueur qui joue
- `afficheJeu()` : fonction pour afficher le plateau du jeu
- `f1(C)` : fonction d'évaluation
- `coupsPossibles(C)` : fonction qui retourne les coups possibles
- `joueLeCoup(C, coup)` : fonction qui permet de jouer un coup
- `estFini(C)` : vérifie si le jeu est fini
- `getGagnant(C)` : retourne le gagnant du jeu

### Morpion

Le jeu est représenté par un tableau de 3x3, si le joueur 1 (resp. joueur 2) à joué dans une case alors la case vaut 1 (2), sinon la case est vide et vaut 0. 

Les fonctions implémentées:
- `coupsPossible(C)` : rend la liste des coups possibles, un coup est représenté par un tuple de i, j;
- `afficheJeu()` : affiche le jeu avec des ronds (O) et des croix (X);
- `f1(C)` : fonction d'évaluation pour le Morpion, retourne la somme des évaluations en ligne, en colonne et en diagonale;
- `eval_ligne(C)`/`eval_colonne(C)`/`eval_diagonale(C)` : fonction d'évaluation en ligne/colonne/diagonale:
    - s'il existe qu'un seul 1 alors la ligne/colonne/diagonale est évalué à 1;
    - s'il existe que deux 1 alors la ligne/colonne/diagonale est évalué à 3;
    - s'il existe qu'un seul 2 alors la ligne/colonne/diagonale est évalué à -2;
    - s'il existe que deux 1 alors la ligne/colonne/diagonale est évalué à -6;
- `joueLeCoup(C, coup)` : retourne la configuration après avoir joué le coup;
- `estFini(C)` : fonction qui vérifie si le jeu est fini;
- `estGagnant(C)` : fonction qui vérifie qui a gagné;


### Allumettes

Le jeu est représenté par une liste de taille g, initialisé par m dans toutes les cases. (g étant le nombre de groupe, m le nombre d'allumettes dans chaque groupe)

Les fonctions implémentées:
- `coupsPossible(C)` : rend la liste des coups possibles, un coup est représenté par un tuple de i, j pour représenter qu'on prend j allumettes du groupe i;
- `afficheJeu()` : affiche combien d'allumettes restes dans chaque groupe
- `f1(C)` : fonction d'évaluation pour le jeu des allumettes, utilise le théorème de Sprague-Grundy pour calculer l'état d'un jeu
On change en binaire le nombre d'allumettes restantes pour chaque groupe et on fait le XOR de tous ces valeurs. Si la valeur est 0 c'est un noyau.
- `joueLeCoup(C, coup)` : retourne la configuration après avoir joué le coup;
- `estFini(C)` : fonction qui vérifie si le jeu est fini;

###  StrategieAleatoire

Une stratégie qui choisi un coup aléatoirement depuis la liste des coups possibles.

### StrategieHumaine

Une stratégie pour débuguer et jouer.

### StrategieMiniMax

Une stratégie qui simule les k coups suivants pour prendre décision d'un coup. (k étant un entier)

Les fonctions implémentées:
- `choisirProchainCoup(C)` : retourne le coup choisi
- `decision(C, listecoup, profondeur, joueur)` : calcule le gain pour chacun des coups possibles et retourne le coup qui rend le plus
- `estimation(C, coup, tour, joueur, profondeur)` : simule les k coups à partir du coup choisi
- `evaluation(C, tour)` : retourne l'évaluation de la configuration donnée

### StrategieAllumettes

Une stratégie optimal pour le jeu des allumettes, utilise le théorème et la fonction de Grundy.

Lors de l'initalisation, il construit l'arbre complètes des états et calcule les valeurs de chaque état selon Grundy.

Les fonctions implémentées:
- `choisirProchainCoup(C)` : retourne le coup qui entre dans le noyau s'il existe, sinon choisi aléatoirement un coup.

## Lancement d'un jeu
pour lancer une partie on utilise la fonction suivante :

`unepartie(nomJeu,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire, Affichage=False)`

on définit aussi des paramètres globaux pour les divers jeux avec m et g pour les Allumettes
et les profondeurs pour les joueurs MinMax.

In [21]:
profondeur1=2
profondeur2=2
g=4
m=7

In [22]:
def unepartie(nomJeu,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire, Affichage=False):
    if(nomJeu=="Morpion"):
        Jeu = Morpion()
    if(nomJeu=="Allumettes"):
        Jeu = Allumettes(g,m)
    else:
        Jeu = Morpion()
    
    if strategie1 == StrategieMiniMax:
        joueur1=strategie1(Jeu, profondeur1)
    else:
        joueur1=strategie1(Jeu)
    if strategie2 == StrategieMiniMax:
        joueur2=strategie2(Jeu, profondeur2)
    else:
        joueur2=strategie2(Jeu)
    conf= Jeu.Config
    while(not Jeu.estFini(conf)):
        Ccopy=conf.copy()
        jCourant=Ccopy["Courant"]
        if(jCourant==1):
            nextCoup=joueur1.choisirProchainCoup(Ccopy)
        else:
            nextCoup=joueur2.choisirProchainCoup(Ccopy)
        Ccopy=Jeu.joueLeCoup(Ccopy,nextCoup)
        conf=Ccopy
        Jeu.Config=conf
        conf=Jeu.Config
        if(Affichage):
            print("TOUR DU JOUEUR "+str(jCourant))
            print("coup joue : ",nextCoup)
            Jeu.afficheJeu()
    
    if(Affichage):
        Jeu.afficheJeu()
    if(not Jeu.egalite):
        winner=Jeu.Config["Courant"]
    else:
        winner=0
    if(Affichage):
        if(not Jeu.egalite):
            print ("Vainqueur J"+str(winner))
        else:
            print ("Match Nul")
    return winner,Jeu.Config

De plus on ajoute une autre fonction afin de lancer plusieurs parties en une.
Cette fonction va effectuer __n x 2__ parties pour vérifier les performances de la strategie1 en joueur 1 et en joueur 2 afin d'éviter des évaluations favorable qu'en commençant en tant que premier joueur.

In [4]:
def nparties (nomJeu,nbparties,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire,AfficheMin=False,AfficheMax=False):
    #Victoires j1,j2
    #  [N,V,D]
    j1=[0,0,0]
    for i in range(0,nbparties):
        win,conf=unepartie(nomJeu,strategie1,strategie2,AfficheMax)
        if(AfficheMin):
            print("partie ",i+1," finie J1")
        j1[win]+=1
    if(AfficheMin):
        print("J1 a fait "+str(j1[1])+"/"+str(nbparties)+" Victoire(s), "+str(j1[2])+"/"+str(nbparties)+" Defaite(s), "+str(j1[0])+"/"+str(nbparties)+" Nul(s) en J1")
    j2=[0,0,0]
    for i in range(0,nbparties):
        win,_=unepartie(nomJeu,strategie2,strategie1,AfficheMax)
        if(AfficheMin):
            print("partie ",i+1," finie J2")
        j2[win]+=1
    if(AfficheMin):
        print("J1 a fait "+str(j2[2])+"/"+str(nbparties)+" Victoire(s), "+str(j2[1])+"/"+str(nbparties)+" Defaite(s), "+str(j2[0])+"/"+str(nbparties)+" Nul(s) en position de J2")
    return j1,j2

Pour tester le temps d'exécution de chaque joueur on utilise la fonction suivante :

`tempExecSurUnepartie(nomJeu,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire,Affichage=False,AffichageTemps=False)`

La fonction va evaluer le temps de la partie et indiquer pour chaque joueur le temps moyen de réflexion.
Dans le cadre d'un joueur utilisant Grundy on donne aussi le temps pris pour générer le graphe des états et calculer les valeurs en fonction de Grundy. 

In [5]:
def tempExecSurUnepartie(nomJeu,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire,Affichage=False,AffichageTemps=False):
    if(nomJeu=="Morpion"):
        Jeu = Morpion()
    if(nomJeu=="Allumettes"):
        Jeu = Allumettes(g,m)
    else:
        Jeu = Morpion()
    
    chrono_Grundy1=0.0
    chrono_Grundy2=0.0
    if strategie1 == StrategieMiniMax:
        joueur1=strategie1(Jeu, profondeur1)
    elif(strategie1== StrategieAllumettes):
        if(nomJeu != "Allumettes"):
            raise Exception("StrategieAllumettes utilisable que dans jeu Allumettes")
        chrono_Grundy_debut1=time()
        joueur1=strategie1(Jeu)
        chrono_Grundy_fin1=time()
        chrono_Grundy1=(chrono_Grundy_fin1-chrono_Grundy_debut1)
    else:
        joueur1=strategie1(Jeu)
    if strategie2 == StrategieMiniMax:
        joueur2=strategie2(Jeu, profondeur2)
    elif(strategie2== StrategieAllumettes):
        if(nomJeu != "Allumettes"):
            raise Exception("StrategieAllumettes utilisable que dans jeu Allumettes")
        chrono_Grundy_debut2=time()
        joueur2=strategie2(Jeu)
        chrono_Grundy_fin2=time()  
        chrono_Grundy2=(chrono_Grundy_fin2-chrono_Grundy_debut2)  
    else:
        joueur2=strategie2(Jeu)
    conf= Jeu.Config
    chrono_List1=[]
    chrono_List2=[]

    chrono_partie_debut=time()
    while(not Jeu.estFini(conf)):
        Ccopy=conf.copy()
        jCourant=Ccopy["Courant"]
        if(jCourant==1):
            ###
            chrono_debut1=time()
            nextCoup=joueur1.choisirProchainCoup(Ccopy)
            chrono_fin1=time()
            chrono_List1.append(chrono_fin1-chrono_debut1)
            ###
        else:
            chrono_debut2=time()
            nextCoup=joueur2.choisirProchainCoup(Ccopy)
            chrono_fin2=time()
            chrono_List2.append(chrono_fin2-chrono_debut2)
        Ccopy=Jeu.joueLeCoup(Ccopy,nextCoup)
        conf=Ccopy
        Jeu.Config=conf
        conf=Jeu.Config
        if(Affichage):
            print("TOUR DU JOUEUR "+str(jCourant))
            Jeu.afficheJeu()
    ###
    chrono_partie_fin=time()
    if(Affichage):
        Jeu.afficheJeu()

    # Selection gagnant
    if(not Jeu.egalite):
        winner=Jeu.Config["Courant"]
    else:
        winner=0
    
    #Traitement temps
    tab1=np.array(chrono_List1)
    tab2=np.array(chrono_List2)
    mean1=np.mean(tab1)
    mean2=np.mean(tab2)

    if(AffichageTemps):
        print("Temps de la partie : ",(chrono_partie_fin-chrono_partie_debut)," secondes")
        print("Nb coups joues : ", len(Jeu.Config["History"])," coups")
        print("Temps moyen de reflexion pour joueur 1 : ",mean1," secondes")
        if((strategie1==StrategieAllumettes) and (nomJeu=="Allumettes") ):
            print("Temps de construction du Grundy pour joueur 1 :", chrono_Grundy1," secondes")
        print("Temps moyen de reflexion pour joueur 2 : ",mean2," secondes")
        if((strategie2==StrategieAllumettes) and (nomJeu=="Allumettes") ):
            print("Temps de construction du Grundy pour joueur 2 :", chrono_Grundy2," secondes")
    #Jeu.afficheJeu()
    return winner,Jeu.Config

On ajoute à cela la fonction ci-dessous

`faceoff(nomJeu,nbparties,strategie1=StrategieAleatoire,listAdv=[StrategieAleatoire],strat1Str="",advStr=["StrategieAleatoire"],AfficheMin=False,AfficheMax=False)`

Cette fonction permet de tester les performances d'un joueur donné face à différents adversaires


In [9]:
def faceoff(nomJeu,nbparties,strategie1=StrategieAleatoire,listAdv=[StrategieAleatoire],strat1Str="",advStr=["StrategieAleatoire"],AfficheMin=False,AfficheMax=False):
    
    for i in range(len(listAdv)):
        adversaire=listAdv[i]
        adversaireNom=advStr[i]
        j1,j2=nparties(nomJeu,nbparties,strategie1,adversaire,AfficheMin,AfficheMax)
        print("+----- Faceoff du ",strat1Str," sur le ",nomJeu," -----+")
        print("+----- ",strat1Str," VS ",adversaireNom, " -----+")
        print(strat1Str, " Victoire(s) : ",j1[1],"/",nbparties," Defaite(s) : ",j1[2],"/",nbparties," Nul(s) : ",j1[0],"/",nbparties," en position de J1")
        print(strat1Str, " Victoire(s) : ",j2[2],"/",nbparties," Defaite(s) : ",j2[1],"/",nbparties," Nul(s) : ",j2[0],"/",nbparties," en position de J2")
        print("+------------------------------------------------+")
    return

## Simulations

### Morpion

In [11]:
game="Morpion"  
listAdversaires= [StrategieAleatoire,StrategieMiniMax]
adversairesNoms=["StrategieAleatoire","StrategieMiniMax"]

#### Aléatoire vs Aléatoire temps d'exec et performances face à Aléatoire et Minimax 2

In [23]:
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire,AffichageTemps=True)

Temps de la partie :  0.0  secondes
Nb coups joues :  9  coups
Temps moyen de reflexion pour joueur 1 :  0.0  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes


In [24]:
nbparties =20
faceoff(game,nbparties,strategie1=StrategieAleatoire,listAdv=listAdversaires,strat1Str="StrategieAleatoire",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieAleatoire  sur le  Morpion  -----+
+-----  StrategieAleatoire  VS  StrategieAleatoire  -----+
StrategieAleatoire  Victoire(s) :  8 / 20  Defaite(s) :  8 / 20  Nul(s) :  4 / 20  en position de J1
StrategieAleatoire  Victoire(s) :  5 / 20  Defaite(s) :  13 / 20  Nul(s) :  2 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieAleatoire  sur le  Morpion  -----+
+-----  StrategieAleatoire  VS  StrategieMiniMax  -----+
StrategieAleatoire  Victoire(s) :  0 / 20  Defaite(s) :  16 / 20  Nul(s) :  4 / 20  en position de J1
StrategieAleatoire  Victoire(s) :  0 / 20  Defaite(s) :  19 / 20  Nul(s) :  1 / 20  en position de J2
+------------------------------------------------+


Le jeu semble à l'avantage du joueur 1 car on peut noter un plus grand nombre de victoire face à un autre joueur aléatoire et surtout un plus grand nombre de nul face à un Minimax qui est déterministe contrairement à l'aléatoire

#### Minimax 2,3,4,5 vs Aléatoire temps d'exec 

In [19]:
profondeur1= 2
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)
print()
profondeur1= 3
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)
print()
profondeur1= 4
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)
print()
profondeur1= 5
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)

Profondeur Minimax =  2
Temps de la partie :  0.05501294136047363  secondes
Nb coups joues :  9  coups
Temps moyen de reflexion pour joueur 1 :  0.011002588272094726  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes

Profondeur Minimax =  3
Temps de la partie :  0.2540559768676758  secondes
Nb coups joues :  7  coups
Temps moyen de reflexion pour joueur 1 :  0.06326401233673096  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes

Profondeur Minimax =  4
Temps de la partie :  1.1762621402740479  secondes
Nb coups joues :  7  coups
Temps moyen de reflexion pour joueur 1 :  0.29406553506851196  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes

Profondeur Minimax =  5
Temps de la partie :  4.252594709396362  secondes
Nb coups joues :  7  coups
Temps moyen de reflexion pour joueur 1 :  1.0631486773490906  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes


Le temps d'exécution augmente avec la profondeur. 

C'est attendu car le nombre de coups à évaluer est plus élevé.

#### Faceoff du Minimax face à l'aléatoire et un Minimax de profondeur equivalente

In [25]:
nbparties =20
profondeur1= 2
profondeur2= 2
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J2
+------------------------------------------------+


In [28]:
profondeur1= 3
profondeur2= 3
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J2
+------------------------------------------------+


In [29]:
profondeur1= 4
profondeur2= 4
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  13 / 20  Defaite(s) :  0 / 20  Nul(s) :  7 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J2
+------------------------------------------------+


On observe ici bien le fait qu'à profondeur égale deux MiniMax feront toujours match nul. On peut supposer que face à l'aléatoire la présence de nuls en J2 vient du fait qu'au morpion le joueur 1 est avantagé car il pose en premier.

#### MiniMax a profondeur différentes

On teste ici pour voir au Morpion si la différence de profondeur entraine des performances différentes en termes de taux de victoires

In [30]:
nbparties =20
profondeur1= 1
profondeur2= 2
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  17 / 20  Defaite(s) :  0 / 20  Nul(s) :  3 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J2
+------------------------------------------------+


In [31]:
nbparties =20
profondeur1= 2
profondeur2= 4
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  19 / 20  Defaite(s) :  0 / 20  Nul(s) :  1 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J2
+------------------------------------------------+


In [32]:
nbparties =20
profondeur1= 4
profondeur2= 6
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  15 / 20  Defaite(s) :  0 / 20  Nul(s) :  5 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Morpion  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  0 / 20  Nul(s) :  20 / 20  en position de J2
+------------------------------------------------+


On observe que face à un autre MiniMax la profondeur des joueurs n'influe pas sur leur chances de victoires.

On aurait pu supposer que cela permette de limiter les nuls face à l'aléatoire mais il n'est pas possible de vérifier cela car l'aléatoire aurait pu "mieux jouer" ou "mal jouer" en fonction des exécutions.

### Jeu des Allumettes

In [39]:
game="Allumettes" 
listAdversaires= [StrategieAleatoire,StrategieMiniMax,StrategieAllumettes]
adversairesNoms=["StrategieAleatoire","StrategieMiniMax","StrategieAllumettes"]

profondeur1 = 2
profondeur2 = 2

#### Aléatoire vs Aléatoire temps d'exec et performances face à Aléatoire et Minimax 2

In [35]:
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieAleatoire,strategie2=StrategieAleatoire,AffichageTemps=True)

Temps de la partie :  0.0  secondes
Nb coups joues :  11  coups
Temps moyen de reflexion pour joueur 1 :  0.0  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes


In [38]:
nbparties =20
faceoff(game,nbparties,strategie1=StrategieAleatoire,listAdv=listAdversaires,strat1Str="StrategieAleatoire",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieAleatoire  sur le  Allumettes  -----+
+-----  StrategieAleatoire  VS  StrategieAleatoire  -----+
StrategieAleatoire  Victoire(s) :  9 / 20  Defaite(s) :  11 / 20  Nul(s) :  0 / 20  en position de J1
StrategieAleatoire  Victoire(s) :  9 / 20  Defaite(s) :  11 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieAleatoire  sur le  Allumettes  -----+
+-----  StrategieAleatoire  VS  StrategieMiniMax  -----+
StrategieAleatoire  Victoire(s) :  7 / 20  Defaite(s) :  13 / 20  Nul(s) :  0 / 20  en position de J1
StrategieAleatoire  Victoire(s) :  12 / 20  Defaite(s) :  8 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieAleatoire  sur le  Allumettes  -----+
+-----  StrategieAleatoire  VS  StrategieAllumettes  -----+
StrategieAleatoire  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
StrategieAleat

Contrairement au jeu du Morpion, la stratégie aléatoire a plus de chance de gagner contre Minimax, mais perd à 100% contre la stratégie optimale.

#### Minimax 2,3,4,5 vs Aléatoire temps d'exec 

In [40]:
profondeur1= 2
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)
print()
profondeur1= 3
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)
print()
profondeur1= 4
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)
print()
profondeur1= 5
print("Profondeur Minimax = ",profondeur1)
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieMiniMax,strategie2=StrategieAleatoire,AffichageTemps=True)

Profondeur Minimax =  2
Temps de la partie :  0.4110908508300781  secondes
Nb coups joues :  11  coups
Temps moyen de reflexion pour joueur 1 :  0.06851514180501302  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes

Profondeur Minimax =  3
Temps de la partie :  9.266072034835815  secondes
Nb coups joues :  15  coups
Temps moyen de reflexion pour joueur 1 :  1.158259004354477  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes

Profondeur Minimax =  4
Temps de la partie :  62.12297034263611  secondes
Nb coups joues :  12  coups
Temps moyen de reflexion pour joueur 1 :  10.353828390439352  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes

Profondeur Minimax =  5
Temps de la partie :  1382.8037004470825  secondes
Nb coups joues :  16  coups
Temps moyen de reflexion pour joueur 1 :  172.85046255588531  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes


#### Faceoff du Minimax face à l'aléatoire et un Minimax de profondeur equivalente

In [41]:
nbparties =20
profondeur1= 2
profondeur2= 2
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  4 / 20  Defaite(s) :  16 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  13 / 20  Defaite(s) :  7 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAllumettes  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 /

In [42]:
profondeur1= 3
profondeur2= 3
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  15 / 20  Defaite(s) :  5 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  8 / 20  Defaite(s) :  12 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAllumettes  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  1 /

On remarque que lorsqu'un Minimax est de profondeur paire, ses performance face à la stratégie optimale sont très mauvaises car c'est dû à la stratégie adverse qui fait que seul des situations perdantes remontent l'arbre du Minimax.
Cependant face à un joueur impair, on observe une possibilité pour le joueur Minimax de gagner face à la stratégie optimale dans le cadre de 4 groupes de 7 allumettes. Cela s'explique par le début en noyau de la stratégie optimale qui se trouve alors dans une situation où elle peut perdre.

#### MiniMax a profondeur différentes

In [43]:
nbparties =20
profondeur1= 1
profondeur2= 2
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  18 / 20  Defaite(s) :  2 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  14 / 20  Defaite(s) :  6 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAllumettes  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 /

In [45]:
nbparties =20
profondeur1= 3
profondeur2= 1
faceoff(game,nbparties,strategie1=StrategieMiniMax,listAdv=listAdversaires,strat1Str="StrategieMiniMax",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAleatoire  -----+
StrategieMiniMax  Victoire(s) :  7 / 20  Defaite(s) :  13 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  6 / 20  Defaite(s) :  14 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieMiniMax  -----+
StrategieMiniMax  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieMiniMax  sur le  Allumettes  -----+
+-----  StrategieMiniMax  VS  StrategieAllumettes  -----+
StrategieMiniMax  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
StrategieMiniMax  Victoire(s) :  2 /

La victoire est absolut pour Minimax en jouant en première, même en changeant la profondeur.

#### Stratégie optimale du jeu des Allumettes
##### StrategieAllumettes vs Aléatoire temps d'exec

In [46]:
winner, configFinale = tempExecSurUnepartie(game,strategie1=StrategieAllumettes,strategie2=StrategieAleatoire,AffichageTemps=True)

Temps de la partie :  0.0  secondes
Nb coups joues :  12  coups
Temps moyen de reflexion pour joueur 1 :  0.0  secondes
Temps de construction du Grundy pour joueur 1 : 5.288050413131714  secondes
Temps moyen de reflexion pour joueur 2 :  0.0  secondes


##### Performances face aux autres joueurs  avec MiniMax 1

In [47]:
nbparties =20
profondeur1= 1
profondeur2= 1
faceoff(game,nbparties,strategie1=StrategieAllumettes,listAdv=listAdversaires,strat1Str="StrategieAllumettes",advStr=adversairesNoms,AfficheMin=False,AfficheMax=False)

+----- Faceoff du  StrategieAllumettes  sur le  Allumettes  -----+
+-----  StrategieAllumettes  VS  StrategieAleatoire  -----+
StrategieAllumettes  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J1
StrategieAllumettes  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieAllumettes  sur le  Allumettes  -----+
+-----  StrategieAllumettes  VS  StrategieMiniMax  -----+
StrategieAllumettes  Victoire(s) :  17 / 20  Defaite(s) :  3 / 20  Nul(s) :  0 / 20  en position de J1
StrategieAllumettes  Victoire(s) :  20 / 20  Defaite(s) :  0 / 20  Nul(s) :  0 / 20  en position de J2
+------------------------------------------------+
+----- Faceoff du  StrategieAllumettes  sur le  Allumettes  -----+
+-----  StrategieAllumettes  VS  StrategieAllumettes  -----+
StrategieAllumettes  Victoire(s) :  0 / 20  Defaite(s) :  20 / 20  Nul(s) :  0 / 20  en position de J1
Str

En remarque bien que comme l'état initial est un noyau il perd toujours contre la stratégie optimale lorsqu'il commence en première et l'inverse en changeant l'ordre.

Mais contre les autres stratégies il a un très grande taux de victoire.